Except where otherwise noted, this content is Copyright (c) 2020, [RTE](https://www.rte-france.com) and licensed under a [CC-BY-4.0 license](https://creativecommons.org/licenses/by/4.0/). <img src="https://mirrors.creativecommons.org/presskit/buttons/80x15/png/by.png" width="100px">

To understand, what is dashboard, [Begin Stochastic](https://www.hadar-simulator.org/tutorial/?name=Begin%20Stochastic) is refactored with it. Go to the bottom of notebook to see dashboard.

<img src="figure.png" width="70%">

We use data generated in the next topic [Workflow](https://www.hadar-simulator.org/tutorial?name=Workflow). Input data representes 10 scenarios with different load and eolien productions. There are also random faults for nuclear and gas. These 10 scenarios are unique. They are 10 random sampling on the *God function* to try to predict more widely network adequacy

In [1]:
import hadar as hd
import numpy as np

In [2]:
def read_csv(name):
    return np.genfromtxt('%s.csv' % name, delimiter=' ').T

In [3]:
line = 2000
study = hd.Study(horizon=168, nb_scn=10)\
    .network()\
        .node('a')\
            .consumption(name='load', cost=10**6, quantity=read_csv('load_A'))\
            .production(name='gas', cost=80, quantity=read_csv('gas'))\
        .node('b')\
            .consumption(name='load', cost=10**6, quantity=read_csv('load_B'))\
        .node('c')\
            .production(name='nuclear', cost=50, quantity=read_csv('nuclear'))\
        .node('d')\
            .consumption(name='load', cost=10**6, quantity=read_csv('load_D'))\
            .production(name='eolien', cost=20, quantity=read_csv('eolien'))\
        .link(src='a', dest='b', cost=5, quantity=line)\
        .link(src='b', dest='c', cost=5, quantity=line)\
        .link(src='c', dest='a', cost=5, quantity=line)\
        .link(src='c', dest='b', cost=10, quantity=line)\
        .link(src='c', dest='d', cost=10, quantity=line)\
        .link(src='d', dest='c', cost=10, quantity=line)\
    .build()

In [4]:
optimizer = hd.LPOptimizer()
res = optimizer.solve(study)

In [5]:
agg = hd.ResultAnalyzer(study, res)
plot = hd.HTMLPlotting(agg=agg, unit_symbol='MW', time_start='2020-06-19', time_end='2020-06-27',
                      node_coord={'a': [1.6264, 47.8842], 'b': [1.9061, 47.9118], 'c': [1.6175, 47.7097], 'd': [1.9314, 47.7090]})

## Dashboard

In [6]:
from hadar_dashboard import dashboard

In [7]:
dashboard(plot)